In [ ]:
#encoding:utf-8
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import pickle
import numpy as np
import os
from sklearn import metrics
import importlib
import module
importlib.reload(module)
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
import codecs
import split
from functools import reduce



def get_top(n,p,probs,data):
    ##n个正例 ，p个负例
    
    
    
    
all_f1= []
all_pre = []
all_recall = []
all_accu = []
for i in range(10):
    split.split()
    split.sampling('../../Data/ntcir_train_ns.txt','../../Data/co-training-data/EN_ntcir_train.txt')
    split.sampling('../../Data/CH_ntcir_train_ns.txt','../../Data/co-training-data/CH_ntcir_train.txt')
    split.process()
    
    
    word2vec_C = pickle.load(open('../../Data/co-training-data/chinese_embedding.pkl', 'rb'))
    word2vec_E = pickle.load(open('../../Data/co-training-data/english_embedding.pkl', 'rb'))
    
    E = module.EnglishModule(word2vec_E, embedding_trainable=True)
    C = module.ChineseModule(word2vec_C,embedding_trainable=True)
    E.cuda()
    C.cuda()
    
    dataEtransC = pickle.load(open('../../Data/co-training-data/CH_ntcir_train.pkl', 'rb'))  ##有标签中文数据（英文数据翻译）
    testC = pickle.load(open('../../Data/co-training-data/CH_emnlp_train.pkl', 'rb'))  ##拿出部分当做中文无标签数据  不分batch
    
    finalC = pickle.load(open('../../Data/co-training-data/CH_emnlp_test.pkl', 'rb'))   ##最终验证数据
    
    dataE = pickle.load(open('../../Data/co-training-data/EN_ntcir_train.pkl', 'rb')) ##有标签英文数据
    testCtransE = pickle.load(open('../../Data/co-training-data/EN_emnlp_train.pkl', 'rb'))  ##当做未标记英文数据 不分batch

    E_param = list(E.parameters())
    C_param = list(C.parameters())
    E_params = filter(lambda x: x.requires_grad, E_param)
    C_params = filter(lambda x: x.requires_grad, C_param)

    Etrain_opt = optim.Adam(E_params, lr=0.001)
    Ctrain_opt = optim.Adam(C_params, lr=0.001)

    epochs = 2
    max_f1 = 0    
    max_preds = []
    for epoch in range(epochs):
        ##首先训练英文分类器
        for i, batch_data in enumerate(dataE):
            E.train(mode=True)
            Etrain_opt.zero_grad()

            textE, emoE, positionE, labelE, _, _ = zip(*batch_data)
            unlabelTextE, unlabelEmoE, unlabelPositionC, _, _, _  = zip(*testCtransE)  #不分batch
                 
            textE, emoE, positionE,labelE = np.array(textE), np.array(list(emoE)), np.array(positionE), np.array(labelE)
            unlabelTextE, unlabelEmoE, unlabelPositionC = np.array(textC), np.array(list(emoC)), np.array(positionC)
        
            batch_size = textE.shape[0]
          
            _textE = Variable(torch.LongTensor(textE).cuda())
            _emoE = Variable(torch.LongTensor(emoE).cuda())
            _posE = Variable(torch.FloatTensor(positionE).cuda())
            _labelE = Variable(torch.LongTensor(np.argmax(labelE, 1)).cuda())
            
            _unlabelTextE = Variable(torch.LongTensor(unlabelTextE).cuda())
            _unlabelEmoE = Variable(torch.LongTensor(unlabelEmoE).cuda())
            _unlabelPositionC = Variable(torch.FloatTensor(unlabelPositionC).cuda())
    
            E_hidden = E.init_hidden(batch_size)
            E_logits,E_probs = E(_textE,_emoE,_posE,E_hidden)
            E_loss = crossentropy(E_logits,_labelE)
            E_loss.backward()
            Etrain_opt.step()
            
            i += 1
            if i%1 == 0:
                preds = []
                pred_probs = []
                test_label = []
                inxs = []
                clause_inxs = []
                all_probs = []
                for j,batch_data in enumerate(testCtransE):
                    F.eval()
                    D.eval()
                    C.eval()    
                    
                    textC, emoC, positionC, labelC, inxsC, clause_inxC  = zip(*batch_data)
                    textC, emoC, positionC,labelC = np.array(textC), np.array(list(emoC)), np.array(positionC), np.array(labelC)
                    batch_size = textC.shape[0]
                  
                    _textC = Variable(torch.LongTensor(textC).cuda())
                    _emoC = Variable(torch.LongTensor(emoC).cuda())
                    _posC = Variable(torch.FloatTensor(positionC).cuda())
                    
                    F_textC,F_emoC = F(textC=_textC,emoC=_emoC,train='False')
                  
                    C_hidden = C.init_hidden(batch_size)
                    C_logits,C_probs = C(init_state = C_hidden,textC = F_textC,emoC = F_emoC, posC = _posC, train='False')
                    
                    pred = list(np.argmax(C_probs.data.cpu().numpy(), 1))  
                    preds.extend(pred)
                    all_probs.extend(list(C_probs.data.cpu().numpy()))
                    test_label.extend(list(np.argmax(labelC, 1)))
                    inxs.extend(inxsC)
                    clause_inxs.extend(clause_inxC)
                    pred_probs.extend(C_probs.data.cpu().numpy()[:, 1])
                ####选出top n正例 以及 top k的反例 加入训练集
                acc = metrics.accuracy_score(np.array(test_label), np.array(preds))
                pre = metrics.precision_score(np.array(test_label), np.array(preds))#, average='macro'
                rec = metrics.recall_score(np.array(test_label), np.array(preds))
                f1 = metrics.f1_score(np.array(test_label), np.array(preds))
                if f1 > max_f1:
                    max_f1 = f1
                    max_acc = acc
                    max_pre = pre
                    max_rec = rec
                    max_epoch = epoch
                    max_preds = all_preds                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
                    file_name = 'expand_f1_'+str(f1) + '_pre_'+str(pre)+'_rec_'+str(rec)+'_epoch_'+str(epoch)+'_'+str(0.7)
                    with open('../Data/Prediction/' + file_name, 'w') as f_w:
                        for inx_i in range(len(inxs)):
                            f_w.write(inxs[inx_i] + ' ' + clause_inxs[inx_i] + ' ' + str(pred_probs[inx_i]) + '\n')
                print ('epoch: %d, accuracy: %f, precision: %f, recall: %f, f1: %f, loss:%f'%(epoch, acc, pre, rec, f1, C_loss.data[0]))
            print ('---max---epoch: %d, accuracy: %f, precision: %f, recall: %f, f1: %f---max---'%(max_epoch, max_acc, max_pre, max_rec, max_f1))
    all_f1.append(max_f1)
    all_pre.append(max_pre)
    all_recall.append(max_rec)
    all_accu.append(max_acc)
print(all_f1)
print(all_pre)
print(all_recall)
print(all_accu)
f1 = reduce(lambda x,y:x+y,all_f1)/10
recall = reduce(lambda x,y:x+y,all_recall)/10
precision = reduce(lambda x,y:x+y,all_pre)/10
accu = reduce(lambda x,y:x+y,all_accu)/10
print(f1,recall,precision,accu)
